<a href="https://colab.research.google.com/github/ronakbarman/cigarette-detection/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/project/models/research/:/content/drive/My Drive/project/models/research/slim/'
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/project/models/research/:/content/drive/My Drive/project/models/research/object_detection/'
!python models/research/object_detection/builders/model_builder_test.py
 
# Change the directory path
 
%cd models/research/object_detection

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
 
from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
 
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops
 
if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')
   
%matplotlib inline
 
from utils import label_map_util
 
from utils import visualization_utils as vis_util

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection
!python xml_to_csv.py

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection
!python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection
!python generate_tfrecord.py --csv_input=images/test_labels.csv  --image_dir=images/test --output_path=test.record

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection
!python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/faster_rcnn_inception_v2_pets.config

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-46857 --output_directory inference_graph

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection
import cv2
vidcap = cv2.VideoCapture('test.mp4')
success,image = vidcap.read()
c = 0
while success:
  vidcap.set(cv2.CAP_PROP_POS_MSEC,(c*100)) #per second 10 as *100
  cv2.imwrite('test_images/' + "image%d.jpg" % c, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  c += 1

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from google.colab.patches import cv2_imshow


# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

#if tf.__version__ < '1.4.0':
  #raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')
  

  
  
# This is needed to display the images.
%matplotlib inline




from utils import label_map_util

from utils import visualization_utils as vis_util




# What model to download.
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/content/drive/My Drive/project/models/research/object_detection/inference_graph' + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/content/drive/My Drive/project/models/research/object_detection/training', 'labelmap.pbtxt')

NUM_CLASSES = 37




detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
    
    
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)




def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)




# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/content/drive/My Drive/project/models/research/object_detection/test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, c) ] 

# Size, in inches, of the output images.
IMAGE_SIZE = (24, 18)




def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

def pixelate(image):
    # Get input size
    height, width, _ = image.shape

    # Desired "pixelated" size
    h, w = (16, 16)

    # Resize image to "pixelated" size
    temp = cv2.resize(image, (w, h), interpolation=cv2.INTER_LINEAR)

    # Initialize output image
    return cv2.resize(temp, (width, height), interpolation=cv2.INTER_NEAREST)


count = 1
for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  #Coordinates of detected objects
  width, height = image.size
  ymin = int(output_dict['detection_boxes'][0][0]*height)
  xmin = int(output_dict['detection_boxes'][0][1]*width)
  ymax = int(output_dict['detection_boxes'][0][2]*height)
  xmax = int(output_dict['detection_boxes'][0][3]*width)
  x,y,w,h = xmin,ymin,xmax-xmin,ymax-ymin
  # Extract ROI
  ROI = image_np[y:y+h, x:x+w]

  # Pixelate ROI
  pixelated_ROI = pixelate(ROI)

  # Paste pixelated ROI back into original image
  image_np[y:y+h, x:x+w] = pixelated_ROI
  plt.imsave('censored_image/image%d.jpg' % count, image_np)
  print('writing image%d' % count)
  count += 1

    

In [ ]:
%cd ~
%cd /content
%cd drive/My Drive/project/models/research/object_detection

import cv2
import numpy as np
import os
from os.path import isfile, join

pathIn= 'censored_image/'
pathOut = 'video.avi'
fps = 10

frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]

for i in range(len(files)):
    filename=pathIn + files[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)

    #inserting the frames into an image array
    frame_array.append(img)

    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

    for i in range(len(frame_array)):
    # writing to a image array
      out.write(frame_array[i])
out.release()